**math.ceil 참고 주소 : https://developer.mozilla.org/ko/docs/Web/JavaScript/Reference/Global_Objects/Math/ceil**

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys

import math
import numpy as np
import pandas as pd
import xlwt

# 필요한 정보를 입력 받기
txt = input('키워드 입력: ')
cnt =int(input('몇건 크롤링: '))
# Math.ceil() 함수는 주어진 숫자보다 크거나 같은 숫자 중 가장 작은 숫자를 integer 로 반환
page_cnt = math.ceil(cnt/10) # 크롤링할 전체 페이지 수

f_name = input('3.txt 형태로 저장할 경로와 파일명을 쓰세요: ')
fc_name = input('4.csv 형태로 저장할 경로와 파일명을 쓰세요: ')
fx_name = input('5.xls 형태로 저장할 경로와 파일명을 쓰세요: ')

# 크롬 웹브라우저를 사용해 웹 브라우저를 실행
s_time = time.time()
path = './chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get('https://korean.visitkorea.or.kr/main/main.html')
time.sleep(2)

# 알림창 닫기
try :
    driver.find_element_by_xpath('//*[@id="safetyStay1"]/div/div/div/button').click()
except :
    print("코로나 창이 없습니다")

# 검색창의 이름을 찾아서 검색어를 입력 후 검색을 진행
driver.find_element_by_id("btnSearch").click()
element = driver.find_element_by_id("inp_search")
element.send_keys(txt)
driver.find_element_by_link_text("검색").click()

키워드 입력: 인천
몇건 크롤링: 100
3.txt 형태로 저장할 경로와 파일명을 쓰세요: D:/Python_Project/portfolio/study_coding/book/완친파 웹크롤러대마왕/인천100.txt
4.csv 형태로 저장할 경로와 파일명을 쓰세요: D:/Python_Project/portfolio/study_coding/book/완친파 웹크롤러대마왕/인천100.csv
5.xls 형태로 저장할 경로와 파일명을 쓰세요: D:/Python_Project/portfolio/study_coding/book/완친파 웹크롤러대마왕/인천100.xls


In [2]:
# 사용자가 입력한 건수와 실제 검색 건수를 비교 후 동기화하기

cnt2=0
time.sleep(2)     # 페이지가 로딩 될 때까지 2초 기다립니다

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
search_cnt_1 = soup.find('div','total_check')
search_cnt_2 = search_cnt_1.find('strong').find('span').get_text()
search_cnt_2 = search_cnt_2.replace(',','')

if cnt > int(search_cnt_2):
    cnt2 = int(search_cnt_2)
else:
    cnt2 = cnt
    
real_page_cnt = math.ceil(cnt2/10)

print('\n')
print('='*80)
print('요청하신 검색 건수는 %s 건이었고' %cnt)
print('해당 키워드로 실제 검색된 글의 건수가 총 %s 건이라서' %search_cnt_2)
print('실제 검색 건수는 %s 건으로 재설정하고' %cnt2)
print('총 %s 페이지의 게시글 수집을 진행하겠습니다.' %real_page_cnt)
print('='*80)
print('\n')



요청하신 검색 건수는 100 건이었고
해당 키워드로 실제 검색된 글의 건수가 총 1074 건이라서
실제 검색 건수는 100 건으로 재설정하고
총 10 페이지의 게시글 수집을 진행하겠습니다.




In [3]:
# 페이지 이동하기 
# 페이지 번호를 바꾸면서 실제 데이터 추출하기

no = 1 # 게시글 번호 카운트용 변수
no2 = [ ] # 게시글 번호 저장용 리스트
contents2 = [ ] # 게시글 내용 저장용 리스트
tags2 = [ ] # 해쉬 태그 정보 저장용 리스트

for x in range(1, real_page_cnt+1): # 1페이지부터 반복 시작
    print('%s 페이지 내용 수집 시작합니다=====================' %x)
    print('\n')
    time.sleep(2) # 페이지 로딩 대기
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul','list_thumType flnon').find_all('div','area_txt')
    
    for i in content_list:
        
        f = open(f_name,'a',encoding='utf-8')
        
        # 게시글 번호 입력
        no2.append(no)
        print('1.번호:',no)
        f.write('1.번호:'+str(no)+'\n')
    
        # 게시글의 내용 추출 및 입력
        contents = i.find('div','tit').get_text( )
        contents2.append(contents)
        print('2.요약내용:',contents.strip())
        f.write('2.요약내용:'+str(contents)+'\n')
    
        # 학습목표 3: 예외 처리 사용하기
        try :
            tag = i.find('p','tag').get_text()
        except AttributeError :
            tag = '태그정보가 없습니다'
            tags2.append(tag)
            print('3.해시태그:',tag.strip())
            print("\n")
            f.write('3.해시태그:'+tag+'\n')
            f.write('\n')
        else :
            tags2.append(tag)
            print('3.해시태그:',tag.strip())
            print("\n")
            f.write('3.해시태그:'+tag+'\n')
            f.write('\n')
            
            f.close()
            
        if no == cnt2 :
            break
    
        no += 1
        
    time.sleep(2)        # 페이지 변경 전 2초 대기 

    x += 1
    
    if x == real_page_cnt+1 :   
        break

    if x == 6 :
        driver.find_element_by_link_text("""다음""").click()
    else :
        driver.find_element_by_link_text("""%s""" %x).click()     

1 페이지 내용 수집 시작합니다=====================


1.번호: 1
2.요약내용: 인천 도심을 즐기다
3.해시태그: #당일코스#1박2일#2박3일#월미도#인천차이나타운#인천대교#인천종합어시장#연안부두#대창반점#추천코스


1.번호: 2
2.요약내용: 인천의 명물빵 여행
3.해시태그: #소래포구#울진대게빵소래본점#인천차이나타운#십리향#송월동동화마을#화마을돌고래피자#수도권#먹거리#도심여행#당일치기여행#바다풍경영화촬영지#복합문화공간#데이트코스#연인과함께#맛코스#당일코스#1박2일#2박3일


1.번호: 3
2.요약내용: 인천테마여행_긴 물길을 따라 이어지는 다채로운 여행 경인아라뱃길
3.해시태그: #경인아라뱃길#인천여행#인천가볼만한곳#정서진#국립생물자원관#시천가람터#아라마루#아라폭포#수향원


1.번호: 4
2.요약내용: 인천 가볼만한 곳. 밤의 낭만과 바다의 설렘이 공존하는 곳, 영종도 야간 드라이브 코스
3.해시태그: #인천가볼만한곳#인천여행#드라이브코스#야간드라이브#밤바다#영종도#카페오라#비치카페라군#인천공항#조개구이


1.번호: 5
2.요약내용: 옛 감성 물씬, 인천 구도심(동인천) 테마여행
3.해시태그: #인천여행#인천여행코스#인천가볼만한곳#배다리헌책방거리#배다리성냥마을박물관#인천문화양조장#애관극장#신포국제시장#인천개항박물관#인천아트플랫폼#11월가볼만한곳#가을여행#가을여행코스#10월가볼만한곳#가을여행의_정석


1.번호: 6
2.요약내용: 인천 차이나타운 & 송월동 동화마을
3.해시태그: #인천가볼만한곳#유네스코세계책의수도#한국근대문학관#역사여행#차이나타운#자유공원#송월동동화마을#인생샷#달동네박물관#체험학습#헌책방골목#서점#17_18한국관광100선#수도권#한국관광100선#인천#인천차이나타운&송월동동화마을


1.번호: 7
2.요약내용: 인천테마여행_매력이 넘치는 섬 강화도! 강화관광플랫폼, 강화풍물시장, 소창체험관, 고려궁지
3.해시태그: #인천여행#강화도#강화도여행#강화관광플랫폼#강화풀물시장#소창체험관#고려궁지


1.번호: 8
2.요약

9 페이지 내용 수집 시작합니다=====================


1.번호: 81
2.요약내용: 인천어린이과학관
3.해시태그: #문화시설#어린이#과학관#인천가볼만한곳#인천어린이과학관


1.번호: 82
2.요약내용: 인천 부평문화원
3.해시태그: #문화시설#인천부평문화원#문화강좌#열린문화공간#친구와 함께


1.번호: 83
2.요약내용: 인천학생교육문화회관
3.해시태그: #인천학생교육문화회관#수도권#친구와함께#사계절#스포츠센터#실내놀이터#문화시설


1.번호: 84
2.요약내용: 인천동구청소년수련관
3.해시태그: #문화시설#인천동구청소년수련관#수도권#청소년수련관#열린문화공간#복합문화공간#문화공간#이색체험#체험프로그램#교육프로그램#체험교육#교육여행


1.번호: 85
2.요약내용: 인천수봉문화회관
3.해시태그: #인천수봉문화회관#공연장#예술#음악회#수도권#아이와함께#친구와함께#연인과함께#데이트코스#체험학습#나들이#힐링#실내여행지#기차여행#어린이날가볼만하곳#서울근교여행#문화시설


1.번호: 86
2.요약내용: 인천통일관
3.해시태그: #문화시설#인천통일관#전시관#역사#체험학습#수도권#북한생활#평화통일


1.번호: 87
2.요약내용: SM면세점 (인천공항점)
3.해시태그: #쇼핑#인천공항#인천공항제1여객터미널#인천공항제2여객터미널#면세점


1.번호: 88
2.요약내용: 인천 동부시장
3.해시태그: #인천동부시장#수도권#전통시장#시장맛집#아이와함께#친구와함께#연인과함께#가족여행#나홀로여행#체험학습#나들이#힐링#먹거리#전통시장투어#서울근교여행#드라이브여행#마을축제#쇼핑


1.번호: 89
2.요약내용: 인천 중앙시장
3.해시태그: #인천중앙시장#수도권#전통시장#시장맛집#아이와함께#친구와함께#연인과함께#가족여행#나홀로여행#나들이#먹거리#전통시장투어#쇼핑


1.번호: 90
2.요약내용: 인천 송현자유시장
3.해시태그: #인천송현자유시장#수도권#전통시장#시장맛집#아이와함께#친구와함께#연인과함께#가족여행#나홀로여행#나들이#먹거리#전통시장투어#서울근교여행#쇼핑



In [4]:
# 출력 결과를 표(데이터 프레임) 형태로 만들기

korea = pd.DataFrame()
korea['번호'] = no2
korea['내용'] = contents2
korea['태그'] = tags2

# 추출결과를 csv 형태로 저장하기
korea.to_csv(fc_name, encoding='utf-8-sig', index=False)

# 추출결과를 엑셀 형태로 저장하기
korea.to_excel(fx_name, index=False)

e_time = time.time() # 검색이 종료된 시점의 timestapm를 지정
t_time = e_time-s_time

print('\n')
print('='*80)
print('크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료했습니다.' %(cnt,no))
print('총 소요시간은 %s 초 입니다' %round(t_time))
print('파일 저장 완료: txt 파일명 : %s' %f_name)
print('파일 저장 완료: csv 파일명 : %s' %fc_name)
print('파일 저장 완료: xls 파일명 : %s' %fx_name)
print('='*80)
print('\n')

driver.close()



크롤링을 요청한 총 100 건 중에서 100 건의 데이터를 수집 완료했습니다.
총 소요시간은 414 초 입니다
파일 저장 완료: txt 파일명 : D:/Python_Project/portfolio/study_coding/book/완친파 웹크롤러대마왕/인천100.txt
파일 저장 완료: csv 파일명 : D:/Python_Project/portfolio/study_coding/book/완친파 웹크롤러대마왕/인천100.csv
파일 저장 완료: xls 파일명 : D:/Python_Project/portfolio/study_coding/book/완친파 웹크롤러대마왕/인천100.xls


